In [1]:
# File parsing and data handling
from pysam import VariantFile
import pandas as pd
import numpy as np

# File handling
import os

# Data dumping
import bz2
import pickle

### Useful informations

In [2]:
# LENGTHS OF CHROMOSOMES
reference = "/mnt/urbe2/disk3/Antoine/19-11-21_VariantCalling_ARC/input/reference.fa"
fai = "/mnt/urbe2/disk3/Antoine/19-11-21_VariantCalling_ARC/input/reference.fa.fai"
lengths = {}
for line in open(fai, 'r') :
    s = line.strip().split()
    lengths[s[0]] = int(s[1])
    
median_coverage = {
    "ancestor": 341,
    "D2A1": 47,
    "D2B3": 264, # has a 50G
    "D2B3_50G": 408,
    "D2C1": 110, # has a 50G
    "D2C1_50G": 137,
    "D2C3": 106, # has a 50G
    "D2C3_50G": 118,
    "D3A1": 49,
    "D3A3": 105,  # has a 50G
    "D3A3_50G": 177,
    "D4A3": 54,
    "D4B4": 144, # has a 50G
    "D4B4_50G":176,
    "D5B3": 73,
    "D5C1": 120, # has a 50G
    "D5C1_50G": 127,
    "D5C3": 156, # has a 50G
    "D5C3_50G": 194,
    "H2A3": 449,
    "H2B4": 122, # has a 50G
    "H2B4_50G": 27, # LOW COVERAGE SAMPLE
    "H2C3": 148,
    "H3A4": 90, # has a 50G
    "H3A4_50G": 150,
    "H3C4": 127, # has a 50G
    "H3C4_50G": 111,
    "H4A4": 409, # has a 50G
    "H4A4_50G": 404,
    "H4C2": 204,
    "H5A2": 54, # has a 50G
    "H5A2_50G": 63,
    "H5A3": 202,
    "H5A4": 126, # has a 50G
    "H5A4_50G": 131,
    "H5C2": 148, # has a 50G
    "H5C2_50G": 165,
    
    "30H_C3_E4":  199,
    "30H_C3_E5":  209,
    "30H_C36_E5": 244,
    "30H_C48_E5": 275,
    
    "30D_C13_E3": 259,
    "30D_C38_E4": 193,
    "30D_C38_E5": 236,
    "30D_C52_E5": 208,
    
    "P0_C9_E4":   186,
    "P0_C9_E5":   174,
    "P0_C27_E5":  242,
    "P0_C40_E5":  192,
    
    "P100_C8_E3": 413,
    "P100_C8_E4": 431,
    "P100_C30_E3":424,
    "P100_C30_E4":490,
    
    "P250_C8_E3": 390,
    "P250_C8_E4": 334,
    "P250_C17_E3":354,
    "P250_C17_E4":435,
    
    "P500_C16_E4":185,
    "P500_C16_E5":187,
    "P500_C18_E3":291,
    "P500_C30_E3":275,
}

### Get informations about packages

Should print:
```raw
2.3.1
4.0
```

In [3]:
print(pd.__version__)
print(pickle.format_version)

2.3.1
4.0


### Parsing files

In [4]:
def get_windows_coverage_information(cov, sample, lengths, median_coverage, num_bins=251) :
    coverage = pd.read_csv(cov, sep="\t", compression="gzip", usecols=range(3), names=["ref", "pos", "cov"], header=None, skiprows=1,)
    coverage = coverage.rename(columns={"ref":"CHR", "pos":"POS", "cov":"COV"})
    coverage = coverage.assign(nCOV=coverage["COV"].div(median_coverage[sample]))
    
    per_contig_dfs = {}
    for ctg in lengths.keys() :
        cdf = coverage.query("CHR == @ctg")
        bins = np.linspace(0, lengths[ctg], num_bins)
        cdf = cdf.assign(BIN=pd.cut(cdf["POS"], bins))
        gdf = cdf.groupby(by="BIN", observed=False).agg(
            { "POS":["first", "last", "mean"],
              "COV":["mean", "std", "min", "max"],
              "nCOV":["mean", "std", "min", "max"],
            }
        )
        per_contig_dfs[ctg] = gdf
    
    with bz2.BZ2File("{}_COV_windows.pandas231.pickle40.pbz2".format(sample), 'wb') as f:
        pickle.dump(per_contig_dfs, f)

def get_windows_AF_information(vcf, sample, lengths, num_bins=251) :
    
    dc = {"CHR":[], "POS":[], "AF":[]}
    
    # AF = max(AD)/sum(AD)
    vcf_in = VariantFile(vcf)  # auto-detect input format
    vcf_in.subset_samples(["ancestor", sample])
    
    for n, rec in enumerate(vcf_in) :
        #if n == 500000 :
        #    break
        if n % 1000000 == 0 :
            print("Elapsed records: {}".format(n))
            
        dc["CHR"].append(rec.chrom)
        dc["POS"].append(rec.pos)
        
        try :
            anc_gt = list(rec.samples["ancestor"]["GT"])
            num_unknown = 0
            for al in anc_gt :
                if al is None :
                    num_unknown += 1                    
            if len(set(anc_gt)) == 1 :
                af = None
            elif len(anc_gt) - num_unknown < 2 :
                af = None
            else :
                sample_ad = rec.samples[sample]["AD"] # sample_ad = (25, 10, 0, 0)
                genotype_ad = [sample_ad[allele_index] for allele_index in anc_gt] # anc_gt = (0, 1)
                # genotype_ad = (25,10)
                if len(genotype_ad) > 1 :
                    af = float(max(genotype_ad)/sum(genotype_ad))
                else :
                    af = None
        except :
            print("Warning: line {} = {}".format(n, rec))
            af = None
            ad = None
        
        dc["AF"].append(af)
        
    df = pd.DataFrame().from_dict(dc)
    
    per_contig_dfs = {}
    
    for ctg in lengths.keys() :
        
        cdf = df.query("CHR == @ctg")
        bins = np.linspace(0, lengths[ctg], num_bins)
        cdf = cdf.assign(BIN=pd.cut(cdf["POS"], bins))
        gdf = cdf.groupby(by="BIN", observed=False).agg(
            {"POS":["first", "last", "mean"], "AF":["mean", "std"],}
        )
        per_contig_dfs[ctg] = gdf
    
    with bz2.BZ2File("{}_AF_windows.pandas231.pickle40.pbz2".format(sample), 'wb') as f:
        pickle.dump(per_contig_dfs, f)

# 1. Read all VCFs

In [ ]:
GR_samples = ['ancestor', 'P100_C8_E4', '30H_C36_E5', 'P250_C8_E3', '30D_C52_E5', '30D_C38_E4', 'P500_C16_E5', 'P100_C30_E4', '30H_C3_E5', 'P0_C27_E5', 'P500_C16_E4', '30D_C13_E3', '30D_C38_E5', 'P0_C9_E5', '30H_C48_E5', 'P250_C17_E4', 'P0_C9_E4', '30H_C3_E4', 'P100_C8_E3', 'P250_C8_E4', 'P500_C30_E3', 'P250_C17_E3', 'P500_C18_E3', 'P0_C40_E5', 'P100_C30_E3']
ME_samples = ['D5C1', 'D2C1', 'H4A4', 'D4B4_50G', 'H5A2', 'D3A3_50G', 'H3C4_50G', 'H5A2_50G', 'H5C2_50G', 'H3A4_50G', 'D4B4', 'H2C3', 'D5C1_50G', 'H5A4', 'H4A4_50G', 'H2A3', 'D2C3_50G', 'H2B4_50G', 'D3A3', 'D2B3', 'H3A4', 'D3A1', 'H5A3', 'D5B3', 'D5C3', 'D5C3_50G', 'D2C3', 'H4C2', 'H5A4_50G', 'D2C1_50G', 'H2B4', 'D4A3', 'H5C2', 'D2A1', 'D2B3_50G', 'H3C4']

GR_bcf = "/mnt/urbe2/disk3/Antoine/19-11-21_VariantCalling_ARC/jointgenotyping/merged.only_het.gets.bcf"
ME_bcf = "/mnt/urbe2/disk3/Antoine/27-10-21_VariantCalling_MA/genotype_allsamples/merged.only_het.gets.bcf"

for sample in GR_samples :
    print("Reading VCF for sample: {}...".format(sample))
    get_windows_AF_information(GR_bcf, sample, lengths)

for sample in ME_samples :
    print("Reading VCF for sample: {}...".format(sample))
    get_windows_AF_information(ME_bcf, sample, lengths)


# 2. Read all Coverages

In [6]:
data_gr = os.listdir("/mnt/urbe2/disk3/Antoine/19-11-21_VariantCalling_ARC/coverage")
data_gr = [os.path.join("/mnt/urbe2/disk3/Antoine/19-11-21_VariantCalling_ARC/coverage", f) for f in data_gr if f.endswith(".cov.gz")]

data_me = os.listdir("/mnt/urbe2/disk3/Antoine/27-10-21_VariantCalling_MA/coverage/")
data_me = [os.path.join("/mnt/urbe2/disk3/Antoine/27-10-21_VariantCalling_MA/coverage/", f) for f in data_me if f.endswith(".cov.gz")]

samples_gr = {os.path.basename(filepath).split(".")[0]:filepath for filepath in data_gr}
samples_me = {os.path.basename(filepath).split(".")[0]:filepath for filepath in data_me}

print("GR files:", len(samples_gr))
print("ME files:", len(samples_me))

# Note: ancestor is read twice now, merging dict to avoid this
all_samples = {k:v for k, v in samples_gr.items()}
for k, v in samples_me.items() :
    if k not in all_samples.keys() :
        all_samples[k] = v

print("All files:", len(all_samples))

GR files: 25
ME files: 37
All files: 61


In [7]:
for sample, coverage_file in all_samples.items() :
    print("Reading COV for sample: {}...".format(sample))
    get_windows_coverage_information(coverage_file, sample, lengths, median_coverage)

Reading COV for sample: P0_C9_E5...
Reading COV for sample: P250_C8_E3...
Reading COV for sample: 30D_C13_E3...
Reading COV for sample: P100_C30_E4...
Reading COV for sample: 30H_C3_E5...
Reading COV for sample: P100_C30_E3...
Reading COV for sample: P250_C17_E3...
Reading COV for sample: 30D_C52_E5...
Reading COV for sample: P500_C16_E4...
Reading COV for sample: 30D_C38_E4...
Reading COV for sample: 30H_C3_E4...
Reading COV for sample: P250_C17_E4...
Reading COV for sample: P0_C27_E5...
Reading COV for sample: P500_C16_E5...
Reading COV for sample: P0_C40_E5...
Reading COV for sample: P500_C30_E3...
Reading COV for sample: P100_C8_E3...
Reading COV for sample: P100_C8_E4...
Reading COV for sample: P250_C8_E4...
Reading COV for sample: 30H_C48_E5...
Reading COV for sample: P0_C9_E4...
Reading COV for sample: ancestor...
Reading COV for sample: 30H_C36_E5...
Reading COV for sample: P500_C18_E3...
Reading COV for sample: 30D_C38_E5...
Reading COV for sample: D5C1...
Reading COV for samp